In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

# %cd /content/drive/MyDrive/IT5006_code/

In [2]:
import torch
import numpy as np
from torch import optim as optim
from cmapss import cmapss
from torch.utils.data import DataLoader
from model import Model, Model_CNN, Model_CNN_feature
from train import Trainer
from tqdm import tqdm

c:\Users\hlxu\.conda\envs\IT5006\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## GPU Check

In [3]:
import torch

if torch.cuda.is_available():
    print("GPU available")
    print("GPU name:", torch.cuda.get_device_name(0))

In [4]:
sets = ['FD001', 'FD002', 'FD003', 'FD004']
# sets = ['FD001']
model_name = 'CNN_feature'
# feature_extract_mode = 'default'
feature_extract_mode = 'tsfresh'

iteration_num = 5
window_size = 30
ori_feature_num = 17

In [5]:
model_class = {
    'CNN': Model_CNN,
    'CNN_feature': Model_CNN_feature,
    # 'LSTM': Model_LSTM,
    # 'LSTM_feature': Model_LSTM_feature,
    'Hybrid': Model
}

In [6]:
def run(iteration_num=10, window_size=30, set_name='FD001', model_name='CNN', feature_extract_mode='default'):

    trainset = cmapss(mode='train',
                    feature_extract_mode=feature_extract_mode,
                    dataset=f'./CMAPSSData/train_{set_name}_processed.csv',
                    window_size=window_size)
    testset = cmapss(mode='test',
                    feature_extract_mode=feature_extract_mode,
                    dataset=f'./CMAPSSData/test_{set_name}_processed.csv',
                    rul_result=f'./CMAPSSData/RUL_{set_name}.txt',
                    window_size=window_size)

    handcrafted_feature_num = trainset.feature_extract().shape[1]
    testset.feature_extract()
    print(f'Feature extraction get {handcrafted_feature_num} handcrafted features')

    train_loader = DataLoader(dataset=trainset, batch_size=100, shuffle=True, num_workers=2)
    test_loader = DataLoader(dataset=testset, batch_size=64, shuffle=False, num_workers=2)
    print('Dataset load successfully')

    best_score_list = []
    best_RMSE_list = []
    
    result_name = f'{set_name}_{model_name}_{feature_extract_mode}'

    for iteration in range(iteration_num):
        print('---Iteration: {}---'.format(iteration + 1))
        model = model_class[model_name](window_size, ori_feature_num, handcrafted_feature_num)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        epochs = 32
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        trainer = Trainer(model=model,
                            model_optimizer=optimizer,
                            print_every=20,
                            epochs=epochs,
                            device=device,
                            result_name=result_name)
        best_score, best_RMSE = trainer.train(train_loader, test_loader, iteration)
        best_score_list.append(best_score)
        best_RMSE_list.append(best_RMSE)

    best_score_list = np.array(best_score_list)
    best_RMSE_list = np.array(best_RMSE_list)
    result = np.concatenate((best_score_list, best_RMSE_list)).reshape(2, iteration_num)
    np.savetxt(f'./result/{result_name}_result.txt', result, fmt='%.4f')

In [7]:
for set_name in sets:
    run(iteration_num = iteration_num, 
        window_size = window_size,
        set_name = set_name,
        model_name = model_name,
        feature_extract_mode = feature_extract_mode)

Run tsfresh feature extraction for train dataset


Processing units: 100%|██████████| 17731/17731 [00:12<00:00, 1380.99unit/s]


Run tsfresh feature extraction for test dataset


Processing units: 100%|██████████| 100/100 [00:00<00:00, 1111.15unit/s]


Feature extraction get 47 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/178, loss(avg. on 20 batches: 0.13266487307846547
batch:40/178, loss(avg. on 20 batches: 0.06599375773221254
batch:60/178, loss(avg. on 20 batches: 0.052328605111688374
batch:80/178, loss(avg. on 20 batches: 0.046980465203523634
batch:100/178, loss(avg. on 20 batches: 0.04258658811450004
batch:120/178, loss(avg. on 20 batches: 0.038990852609276774
batch:140/178, loss(avg. on 20 batches: 0.035104424133896826
batch:160/178, loss(avg. on 20 batches: 0.03522915206849575
test result: score: 9161.501953125, RMSE: 23.727205276489258
Epoch: 2
batch:20/178, loss(avg. on 20 batches: 0.03148284684866667
batch:40/178, loss(avg. on 20 batches: 0.03167982660233974
batch:60/178, loss(avg. on 20 batches: 0.02769354050979018
batch:80/178, loss(avg. on 20 batches: 0.027674633730202913
batch:100/178, loss(avg. on 20 batches: 0.026771552674472333
batch:120/178, loss(avg. on 20 batches: 0.026025908

...

Feature extraction get 20 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/219, loss(avg. on 20 batches: 0.35338819250464437
batch:40/219, loss(avg. on 20 batches: 0.09963536523282528
batch:60/219, loss(avg. on 20 batches: 0.07192719988524914
batch:80/219, loss(avg. on 20 batches: 0.05858386512845755
batch:100/219, loss(avg. on 20 batches: 0.048949699848890305
batch:120/219, loss(avg. on 20 batches: 0.04132747687399387
batch:140/219, loss(avg. on 20 batches: 0.03830241439864039
batch:160/219, loss(avg. on 20 batches: 0.03708504401147365
batch:180/219, loss(avg. on 20 batches: 0.03457595389336347
batch:200/219, loss(avg. on 20 batches: 0.034643895272165534
test result: score: 14809.591796875, RMSE: 28.845855712890625
Epoch: 2
batch:20/219, loss(avg. on 20 batches: 0.03098738519474864
batch:40/219, loss(avg. on 20 batches: 0.030088116787374018
batch:60/219, loss(avg. on 20 batches: 0.02945572240278125
batch:80/219, loss(avg. on 20 batches: 0.02890487676

Processing units: 100%|██████████| 54028/54028 [00:22<00:00, 2407.63unit/s]


Run tsfresh feature extraction for test dataset


Processing units: 100%|██████████| 248/248 [00:00<00:00, 2289.52unit/s]


Feature extraction get 20 handcrafted features
Dataset load successfully
---Iteration: 1---
Epoch: 1
batch:20/541, loss(avg. on 20 batches: 0.8596824482083321
batch:40/541, loss(avg. on 20 batches: 0.2375029768794775
batch:60/541, loss(avg. on 20 batches: 0.09675362855195999
batch:80/541, loss(avg. on 20 batches: 0.07562231402844191
batch:100/541, loss(avg. on 20 batches: 0.07064470779150725
batch:120/541, loss(avg. on 20 batches: 0.07034699507057667
batch:140/541, loss(avg. on 20 batches: 0.061201229318976404
batch:160/541, loss(avg. on 20 batches: 0.056885441020131114
batch:180/541, loss(avg. on 20 batches: 0.05389976855367422
batch:200/541, loss(avg. on 20 batches: 0.052183865755796435
batch:220/541, loss(avg. on 20 batches: 0.05243302695453167
batch:240/541, loss(avg. on 20 batches: 0.05008516777306795
batch:260/541, loss(avg. on 20 batches: 0.05091069769114256
batch:280/541, loss(avg. on 20 batches: 0.051008071005344394
batch:300/541, loss(avg. on 20 batches: 0.05108942613005638
b